In [603]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re
import time
# Aqui comence con los cambios, quite algunas librerias que no se estan usando [GM]

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [604]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [605]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))
ahora_format = "%Y-%m-%d"'T'"%H:%M:%S"
ahora = str(now.strftime(ahora_format))
ahora_cdd = str(now.strftime("%Y-%m-%d"' '"%H:%M:%S"))
fechaAhora = str(now.strftime("%Y%m%d%H%M%S"))
datos_logs =""
#fechaAhora

### Definiendo indice principal con fecha de hoy
* 1.- Primer detalle (aqui se deberia de quitar esa fecha_hoy ? )
*     [ indice parameter= "dev-mintic-concat" ]   [GM]



In [ ]:
indice = parametros.gestion_mintic_concat_index

### Función para generar JSON compatible con ES

In [608]:
def filterKeys(document, use_these_keys):
    return {key: document.get(key) for key in use_these_keys }

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [609]:
# aqui se busca la fecha ultima de control de Estado Cambuim  
                   # tengo una pregunta:  porque usa 
                   # parametros.mintic_control=dev-control_ejecucion_mintic
                   # en vez de  indice = parametros.mintic_concat_index que es = dev-mintic-concat-2021.06.24
                   # me imagino que es donde guardan todos los controles de las ejecuciones  * devuelvo el cambio *
total_docs = 1
try:
    response = es.search(
        index=  parametros.mintic_control,     # ***  quite esto, creo que no va, pero lo devolvi,habia puesto indice
        body={
           "_source": ["gestion.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia_gestion_estado_cdd"
                    }
                  }
              ]
            }
          }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion_bmc = doc["_source"]['gestion.fechaControl']
except:
    response["hits"]["hits"] = []
if response["hits"]["hits"] == []:    
    fecha_ejecucion_cdd = '2021-04-01 00:00:00'   # el formato tiene que ser solo numeros, asi aparecen los registros[GM]
    
print("ultima fecha para control de ejecucion gestion_estado_cdd: ",fecha_ejecucion_cdd)

ultima fecha para control de ejecucion gestion_estado_cdd:  2021-04-01 00:00:00


In [155]:
#parametros.mintic_control    #  Estoy trabajando en DEV :  'dev-control_ejecucion_mintic'


In [610]:
total_docs = 1
try:
    response = es.search(
        index= parametros.mintic_control,
        body={
           "_source": ["gestion.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia_gestion_estado_bmc"
                    }
                  }
              ]
            }
          }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion_bmc = doc["_source"]['gestion.fechaControl']
except:
    response["hits"]["hits"] = []
if response["hits"]["hits"] == []:
    #fecha_ejecucion_bmc = '2021-04-01T00:00:00'   # 
    fecha_ejecucion_bmc = '20210401000000'   # el fomrato tiene que ser solo numeros, asi aparecen los registros[GM]
    fecha_ejecucion_cdd = '2021-04-01 00:00:00'   # el fomrato tiene que ser solo numeros, asi aparecen los registros[GM]
    
print("ultima fecha para control de ejecucion gestion_estado_bmc: ",fecha_ejecucion_bmc)

ultima fecha para control de ejecucion gestion_estado_bmc:  20210701002000


In [611]:
total_docs = 1
try:
    response = es.search(
        index= parametros.mintic_control,
        body={
           "_source": ["gestion.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                              "field":"jerarquia_gestion_estado_incidentes"
                    }
                  }
              ]
            }
          }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion_inc = doc["_source"]['gestion.fechaControl']
except:
    response["hits"]["hits"] = []
if response["hits"]["hits"] == []:
    fecha_ejecucion_inc = '2021-04-01T00:00:00'
print("ultima fecha para control de ejecucion gestion_estado_incidentes:",fecha_ejecucion_inc)



ultima fecha para control de ejecucion gestion_estado_incidentes: 2021-04-01T01:30:00


### 1.- leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede.)

In [641]:
t1=time.time()

total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob','energiadesc'
                           ,'nombreSede','latitud', 'longitud','id_Beneficiario','COD_ISO','codDanesede',
                           'cod_servicio','codDaneMuni','nombre_centro_pob','codCentroPoblado','codDaneInstitucionEdu',
                           'tipoSitio','detalleSitio','energia','region','matricula','DDA','grupoDesc','estadoInstalacion']
        },
        size=total_docs
    )
    #print(es.info())  antes era:
    # ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob','energiadesc'
    #                       ,'nombreSede','latitud', 'longitud','id_Beneficiario','COD_ISO','codDanesede'
    #                       ,'DDA','estadoInstalacion','nombreInstitucionEd','matricula','municipioPDET']
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])

#     datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
    
    datos_semilla = pd.DataFrame([x["_source"] for x in elastic_docs])

except Exception as e:
    print(e.message)
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()

t2=time.time()

In [643]:
datos_semilla.columns

Index(['nombreSede', 'DDA', 'estadoInstalacion', 'latitud', 'COD_ISO',
       'energia', 'nombre_departamento', 'codCentroPoblado', 'longitud',
       'codDanesede', 'tipoSitio', 'codDaneMuni', 'nombre_centro_pob',
       'site_id', 'matricula', 'energiadesc', 'grupoDesc', 'cod_servicio',
       'region', 'detalleSitio', 'nombre_municipio', 'id_Beneficiario',
       'codDaneInstitucionEdu'],
      dtype='object')

In [613]:
def get_location(x,y='lat'):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None) and (str(x)!=''):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return '4.596389' if y=='lat' else '-74.074639'

In [614]:
datos_semilla['latitud'] = datos_semilla['latitud'].apply(lambda x:get_location(x,'lat'))
datos_semilla['longitud'] = datos_semilla['longitud'].apply(lambda x:get_location(x,'lon'))
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['gestion.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['gestion.location']=datos_semilla['gestion.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

### 2.- Leyendo datos de Cambium-Device-Devices

In [615]:
# Se extraen los datos de cambium-devicedevices,  'cambium-devicedevices'

def traerDDevices(fecha_max,fecha_tope):
    total_docs = 10000
    try:
        response = es.search(
            index= parametros.cambium_d_d_index,
            body={
                "_source": ["registration_date","mac","ip","ap_group","site_id","status"]
                ,"query":{
                    "bool": {
                      "filter": [
                        {
                          "range": {
                            "registration_date": {
                              "gte": fecha_max,
                              "lt": fecha_tope
                            }
                          }
                        }
                      ],

                    }
                }
            },
            size=total_docs
        )
        elastic_docs = response["hits"]["hits"]
#         fields = {}
#         for num, doc in enumerate(elastic_docs):
#             source_data = doc["_source"]
#             for key, val in source_data.items():
#                 try:
#                     fields[key] = np.append(fields[key], val)
#                 except KeyError:
#                     fields[key] = np.array([val])

#         datos_DDevices  = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
        
        datos_DDevices = pd.DataFrame([x["_source"] for x in elastic_docs])
        
        datos_DDevices  = datos_DDevices.rename(columns={'ap_group':'gestion.ptos_acceso'})
        datos_DDevices  = datos_DDevices.dropna(subset=['site_id'])
        return datos_DDevices 
    except Exception as e:
#         print(e.message)
        return pd.DataFrame()
    

## 3.- Se leen datos de BMC

* Se acota por el rango de fecha
* BMC trae las alertas generadas por sitio

In [616]:
def traerBMC(fecha_max,fecha_tope):
    total_docs = 1000
    try:
        response = es.search(
            index= parametros.bmc_index_1,
            body={
                "_source": ["*"]
                ,"query":{
                    "bool": {
                      "filter": [
                        {
                          "range": {
                            "fechahora": {
                              "gte": fecha_max,
                              "lt": fecha_tope
                            }
                          }
                        }
                      ],

                    }
                }
            },
            size=total_docs
        )
        elastic_docs = response["hits"]["hits"]
        
#         print(elastic_docs)
        
        fields = {}
        for num, doc in enumerate(elastic_docs):
            source_data = doc["_source"]
            for key, val in source_data.items():
                try:
                    fields[key] = np.append(fields[key], val)
                except KeyError:
                    fields[key] = np.array([val])

        datos_BMC = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
        
        
#         datos_BMC = pd.DataFrame([x["_source"] for x in elastic_docs]) #Los dataframe son diferentes..
        
        datos_BMC = datos_BMC.rename(columns={'mc_location':'site_id'})
        datos_BMC = datos_BMC.dropna(subset=['site_id'])
        return datos_BMC
    except Exception as e:
#         print(e)
        return pd.DataFrame()
    

### 4.- Leyendo indice servicemanager-incidentes

Se lee la información de los De Service Manager. En la lectura se traen todas las interacciones reportadas por servicemanager, con la información de la misma.

* clr_bmc_location, es la llave para cruzar con cada centro de conexión.
* product_type, es el detalle del ticket.
* clr_bmc_host, estado la IP
* assignment, es el sujeto asignado al ticket
* contact_name, usuario del ticket
* open_time y resolved_time, son las fecha de apertura y cierre del ticket
* resolution, respuesta al ticket
* number, numero del ticket
* source, fuente del ticket(email o telefono)
* severity, gravedad del ticket
* category y subcategory, clasificacion dada por mintic

In [617]:
def traeSMIncidentes(fecha_max,fecha_tope):
    total_docs = 100
    try:
        response = es.search(
            index= parametros.sm_incidentes,
            body={
                "_source": ["clr_bmc_location","resolved_time"]  # clr_bmcdatevent= fecha del evento, open_time=fecha del ticket
                ,"query": {
                    "bool": {
                      "filter": [
                        {
                          "range": {
                            "@timestamp": {
                              "gte": fecha_max,
                              "lt": fecha_tope
                            }
                          }
                        }
                      ], 
                      "must_not": [
                        {
                          "exists": {
                            "field": "gestion.fechaCierre"
                          }
                        }
                      ]
                    }
                  }
            },
            size=total_docs
        )
        elastic_docs = response["hits"]["hits"]
        fields = {}
        for num, doc in enumerate(elastic_docs):
            source_data = doc["_source"]
            for key, val in source_data.items():
                try:
                    fields[key] = np.append(fields[key], val)
                except KeyError:
                    fields[key] = np.array([val])

        datos_SM_incidents = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
        
#         datos_SM_incidents = pd.DataFrame([x["_source"] for x in elastic_docs])
        
        datos_SM_incidents = datos_SM_incidents.rename(columns={'clr_bmc_location':'site_id'})
        
        if datos_SM_incidents.empty:
            datos_SM_incidents=pd.DataFrame(columns=['site_id','resolved_time'])
        
        return datos_SM_incidents

    except Exception as e:
#         print(e)
        return pd.DataFrame(columns=['site_id','resolved_time'])

Realizando bucle hasta conseguir datos de servicemanager-incidentes o hasta la fecha actual para realizar la carga de datos

In [618]:
def drop_invalid_values(df, fields=[], values = ['NA','',np.NaN,None],operator='or'):
    fields = df.columns if fields==[] else fields
    if operator=='or':
        freduce = np.logical_or.reduce
    elif operator=='and':
        freduce = np.logical_and.reduce
    else:
        return
    
    df.drop(df[freduce([df[c].isin(values) for c in fields])].index,inplace=True)

In [619]:
fec1="2021-04-01 00:00:00"

datos_CDD  = traerDDevices(fec1 , ahora_cdd)

datos_logs= "Buscado en cambium-DD desde  " + fec1 +" hasta " + ahora_cdd

if datos_CDD is None or datos_CDD.empty:
    datos_logs = datos_logs +"\n No trajo datos en este rango de fecha. "
    pass
else:
    
    drop_invalid_values(datos_CDD,['gestion.ptos_acceso','site_id','status'])
    
    datos_CDD  = datos_CDD.dropna(subset=['site_id'])

    datos_CDD = datos_CDD.drop(datos_CDD[(datos_CDD['site_id']=='')].index)
    datos_logs= datos_logs + "\n total reg.: " + str(datos_CDD["site_id"].size) + "    viene con (registration_date,gestion.ptos_acceso,ip,site_id,mac,status)"

    #datos_CDD2 = datos_CDD[["site_id","registration_date", "status", "gestion.ptos_acceso"]]
    datos_CDD2 = datos_CDD[["site_id", "status"]]    #datos_CDD2 = datos_CDD2[["site_id","registration_date", "status", "gestion.ptos_acceso"]].groupby(["site_id", "status", "gestion.ptos_acceso"]).agg(['max']).reset_index()
    datos_CDD2 = datos_CDD2[["site_id","status"]].groupby(["site_id"]).agg(['count']).reset_index()

    datos_CDD2.columns = datos_CDD2.columns.droplevel(1)       
    datos_logs = datos_logs +"\n se quitan los repetidos status=(offline, online, onboaring) total reg.: " + str(datos_CDD2["site_id"].size)


#print (datos_logs)    # ojoooooo  cuando se pase a Produccion se quita esta linea

In [620]:
fecha_max_bmc = fecha_ejecucion_bmc                
#fecha_max_bmc = "20210401200000"  aqui nos dimos cuenta que el formato es distinto [GM]

fecha_tope_bmc = (datetime.strptime(fecha_max_bmc, "%Y%m%d%H%M%S")+timedelta(minutes=59)-timedelta(seconds=1)).strftime("%Y%m%d%H%M%S")

#fecha_tope_bmc =  "20210601200000"
datos_BMC = traerBMC(fecha_max_bmc,fecha_tope_bmc)

In [621]:
datos_logs = datos_logs +"\n Busco en BMC desde " + fecha_max_bmc+", hasta "+ fecha_tope_bmc

fecha_max_inc = fecha_ejecucion_inc
fecha_tope_inc = (datetime.strptime(fecha_max_inc, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=59)-timedelta(seconds=1)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
#fecha_tope_inc = "2021-06-01T20:00:00"
datos_SM_incidents = traeSMIncidentes(fecha_max_inc,fecha_tope_inc)

In [622]:
datos_logs = datos_logs +"\n Busco en SMIncidentes desde " + fecha_max_inc+ ", hasta "+ fecha_tope_inc +" y lo recorro hasta hoy "

if (datos_BMC is None or datos_BMC.empty) & (datos_SM_incidents is None or datos_SM_incidents.empty):
    while (datos_BMC is None or datos_BMC.empty) and ((datetime.strptime(fecha_max_bmc[0:8], '%Y%m%d').strftime("%Y%m%d%H%M%S")) < str(now.strftime("%Y%m%d%H%M%S"))):
        fecha_max_bmc = (datetime.strptime(fecha_max_bmc, "%Y%m%d%H%M%S")+timedelta(minutes=59)).strftime("%Y%m%d%H%M%S")
        fecha_tope_bmc = (datetime.strptime(fecha_tope_bmc, "%Y%m%d%H%M%S")+timedelta(minutes=59)).strftime("%Y%m%d%H%M%S")
        datos_BMC = traerBMC(fecha_max_bmc,fecha_tope_bmc)
        datos_SM_incidents = traeSMIncidentes(fecha_max_inc,fecha_tope_inc)        
else:
    pass
#datos_BMC
#datos_BMC2 = traerBMC("2021-05-01T00:00:00","2021-06-22T00:10:00")
#datos_BMC2 = traerBMC("20210401010000","20210607200000")
# datos_SM_incidents = traeSMIncidentes("2021-05-01T00:00:00","2021-06-22T00:10:00")

In [623]:
datos_SM_incidents.fillna({'resolved_time':'0'},inplace=True)  ##  ojooooooo aqui hay que ponerlo en 0  // 2021-04-01T05:50:00
datos_SM_incidents_liberado = datos_SM_incidents[ (datos_SM_incidents["resolved_time"]!="0") ]
datos_SM_incidents_liberado=datos_SM_incidents_liberado[['site_id','resolved_time'
                                       ]].groupby(['site_id']).agg(['max']).reset_index()
datos_SM_incidents_liberado.columns = datos_SM_incidents_liberado.columns.droplevel(1)

In [624]:
nbmc=0
if datos_BMC is None or datos_BMC.empty:
    nbmc=0
else: 
    nbmc=datos_BMC["site_id"].size

datos_BMC2 = datos_BMC.groupby(['site_id','fechahora','itsm_incident_id']).count().reset_index()  # no se para que es esta linea , solo veo que pone los demas campos en 0 , y creo que deberian de estar en 1
datos_BMC2 = datos_BMC2.rename(columns={'nombreSede':'gestion.estadoCentro'})

In [625]:
datos_BMC2.drop(columns=['severity','mc_host','@timestamp','mc_modhist','itsm_incident_status','mc_client_address','CLASS','mc_original_severity'],inplace=True)
#datos_BMC2 = datos_BMC2.rename(columns={'fechahora' : 'estado'})

In [626]:
datos_BMC = datos_BMC2
datos_BMC.fillna({'itsm_incident_id':'0'},inplace=True)
datos_BMC.replace('','0',inplace=True)


datos_BMC=datos_BMC[['site_id','fechahora','itsm_incident_id'
                                       ]].groupby(['site_id']).agg(['max']).reset_index()

datos_BMC.columns = datos_BMC.columns.droplevel(1)

In [627]:
bmcX =[]
smi=0
if datos_SM_incidents_liberado is None or datos_SM_incidents_liberado.empty:
    smi=0
else: 
    smi=datos_SM_incidents_liberado["site_id"].size
    try:
        bmcX  = pd.merge(datos_BMC, datos_SM_incidents_liberado , on=['site_id'],how='left')
    except:
        bmcX = pd.DataFrame()

    
datos_logs = datos_logs +"\n  total BMC: "+ str(nbmc) +"  y Total  SMIncidentes: " + str(smi)
#print (datos_logs)    


if len(bmcX) >0:    #aqui se liberan los que vienen en SMI
    bmcX.fillna({'resolved_time':'10a'},inplace=True)
    datos_BMC = bmcX[bmcX["resolved_time"]=="10a"]
    datos_BMC.drop(columns=['resolved_time'],inplace=True)

#datos_BMC

## Lectura de estados
* estos estados guardan el ultimo estado "offline" en que estuvo el centro 


In [628]:
total_docs = 10000
try:
    response = es.search(
        index= "edo_sitio-" + indice,
        body={
               "_source": ['site_id','fechahora','itsm_incident_id']
        },
        size=total_docs
    )    
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    edo_sitio = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
    
    if edo_sitio.empty:
        edo_sitio = pd.DataFrame(columns=['site_id','fechahora','itsm_incident_id'])

except Exception as e:
#     print(e.message)
    edo_sitio = pd.DataFrame(columns=['site_id','fechahora','itsm_incident_id'])
    
edo_sitio["fechahora"] = fechaAhora
#edo_sitio["itsm_incident_id"]    = "0"

#edo_sitio    

In [629]:
try:
    concat0 = pd.merge(datos_semilla, datos_CDD2 , on=['site_id'],how='inner')
except Exception as e:
    print(e.message)
    concat0 = pd.DataFrame()


datos_logs = datos_logs +"\n se cruza con SEMILLA  total reg.: " + str(concat0["site_id"].size)

Haciendo merge entre semilla e incidentes

In [630]:
# esto es el deber ser, lo politicamente correcto 
completo =[]
todos_off =[]
if smi==0:     # si no hay incidentes, es decir centros habilitados 
    #entonces cruzo Semilla con los estados-BMC con left para que me queden los demas online
    if nbmc>0:
        # para colocar todos los offline cruzo BMC con edo_sitio y alli obtengo todos con un outer
        todos_off = pd.merge(edo_sitio, datos_BMC, on=['site_id'],how='outer')
        todos_off["gestion.estadoCentro"]="DESCONECTADOS"
        todos_off.fillna({'fechahora_x':''},inplace=True)
        todos_off.fillna({'fechahora_x':''},inplace=True)
        
        todos_off.fillna({'itsm_incident_id_x':'0'},inplace=True)
        todos_off.fillna({'itsm_incident_id_y':'0'},inplace=True)
        todos_off.fillna({'itsm_incident_id':'0'},inplace=True)
        todos_off["itsm_incident_id"] ='0'
        todos_off["fechahora"] = fechaAhora
        
        #todos_off["fechahora"]  = todos_off["fechahora_y"] + todos_off["fechahora_x"]
        for i in todos_off.index: 
            if todos_off["itsm_incident_id_x"][i]=='0':
                todos_off["itsm_incident_id"][i] = todos_off["itsm_incident_id_y"][i]
            if todos_off["itsm_incident_id_y"][i]=='0':
                todos_off["itsm_incident_id"][i] = todos_off["itsm_incident_id_x"][i]
                
            if todos_off["fechahora_x"][i]=='':
                todos_off["fechahora"][i] = todos_off["fechahora_y"][i]
            if todos_off["fechahora_y"][i]=='':
                todos_off["fechahora"][i] = todos_off["fechahora_x"][i]                             
        
        
        todos_off.drop(columns=['fechahora_x','fechahora_y','itsm_incident_id_x','itsm_incident_id_y'],inplace=True)        
        
        # ahora lo cruzo con semilla 
        print ("  ...... SMI=" , smi, "  y BMC= ", nbmc)
        datos_logs = datos_logs +"\n\n\n en el cruce de los offline que son datos_BMC con edo_sitio tipo outer  total reg.: " + str(todos_off["site_id"].size)
        completo  = pd.merge(concat0, todos_off, on=['site_id'],how='left')
else:
    completo  = concat0
    completo["gestion.estadoCentro"]="CONECTADOS"
    # si hay incidentes es decir se liberaron los centros digitales
    # 

#print (datos_logs)
#completo.to_excel("cdd_completo2.xlsx")        

completo['gestion.estadoCentro'].fillna('CONECTADOS',inplace=True)


#completo 
conectados = 0
desconectados = 0


bb = completo[(completo["gestion.estadoCentro"]=="DESCONECTADOS")].reset_index()
aa = completo[(completo["gestion.estadoCentro"]!="DESCONECTADOS")].reset_index()

conectados = aa["site_id"].size
desconectados = bb["site_id"].size
totales = desconectados + conectados
print (" Conectados: " , conectados , " desconectados: ", desconectados , " total: " , totales)
#i=0
#for index2 in todos_off:    
 #   print (i," reg. site_id: " )
 #   print (index2)
 #   i=i+1
    # df.loc[df['A'] > 2, 'B'] = new_val    ejemplo 

if len(todos_off)>0:
    print ("tienes registro .. todo_off")
else:    
    print ("NO  NOOOOO tienes registro .. todo_off")
    

  ...... SMI= 0   y BMC=  11
 Conectados:  1181  desconectados:  12  total:  1193
tienes registro .. todo_off


### Realizando inserción en ultimo estado del sitio

In [631]:
use_these_keys = ['site_id','gestion.estadoCentro','fechahora', 'itsm_incident_id']
def doc_generator(df,use_these_keys):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": "edo_sitio-" + indice, 
                "_id": f"{'Estado-'+str(document['site_id'])}",
                "_source": filterKeys(document,use_these_keys),
            }
if len(todos_off)>0:        
    salida = helpers.bulk(es, doc_generator(todos_off,use_these_keys))
    print("Fecha: ", now,"- Ult.Estado de los Centros Desconectados  insertadas en indice edo_sitio:",salida[0])


Fecha:  2021-07-15 11:26:14.765854 - Ult.Estado de los Centros Desconectados  insertadas en indice edo_sitio: 13


In [86]:
# es.indices.delete(index="edo_sitio-" + parametros.mintic_concat_index, ignore=[400, 404])
# aqui elimino todo el indice para poder ingresar los nuevos registros con los campos nuevos  GM

Actualizando fecha de control de ejecución

In [632]:
fecha_ejecucion_inc = (datetime.strptime(fecha_max_inc, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")[0:15] + '0:00'    
#fecha_ejecucion_inc = '2021-05-01T00:00:00'

if fecha_ejecucion_inc > str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00':
    fecha_ejecucion_inc = str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00'
response = es.index(
        index = parametros.mintic_control,
        id = 'jerarquia_gestion_estado_incidentes',
        body = { 'jerarquia_gestion_estado_incidentes': 'gestion_estado_incidentes','gestion.fechaControl' : fecha_ejecucion_inc}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion_inc)


actualizada fecha control de ejecucion: 2021-04-01T01:40:00


In [633]:
fecha_ejecucion_bmc = (datetime.strptime(fecha_max_bmc, "%Y%m%d%H%M%S")+timedelta(minutes=10)).strftime("%Y%m%d%H%M%S")[0:11] + '000'    
#fecha_ejecucion_bmc = '20210501000000'

if fecha_ejecucion_bmc > str(now.strftime("%Y%m%d%H%M%S"))[0:11] + '000':
    fecha_ejecucion_bmc = str(now.strftime("%Y%m%d%H%M%S"))[0:11] + '000'
response = es.index(
        index = parametros.mintic_control,
        id = 'jerarquia_gestion_estado_bmc',
        body = { 'jerarquia_gestion_estado_bmc': 'gestion_estado_bmc','gestion.fechaControl' : fecha_ejecucion_bmc}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion_bmc)


actualizada fecha control de ejecucion: 20210701003000


## 10.- insercion en el indice


In [634]:
# primero INSERTAR FINAL
completo.fillna({'fechahora':fechaAhora},inplace=True)
completo.fillna({'gestion.estadoCentro':'CONECTADOS'},inplace=True)
completo.fillna({'itsm_incident_id':0},inplace=True)

try:
    completo = completo.rename(columns={'id_Beneficiario' :  'gestion.estado.id_Beneficiario'
                                                                ,'nombreSede':'gestion.estado.nombreSede'
                                                                ,'site_id':'gestion.estado.site_id'
                                                                ,'nombre_departamento':'gestion.estado.dptoGestion'
                                                                ,'nombre_municipio':'gestion.estado.muniGestion'
                                                                ,'nombre_centro_pob':'gestion.estado.nombre_centro_pob'
                                                                ,'COD_ISO':'gestion.estado.COD_ISO'
                                                                ,'codDanesede':'gestion.estado.codDanesede'
                                                                ,'energiadesc':'gestion.estado.energiadesc'
                                                                ,'DDA':'gestion.estado.DDA'
                                                                ,'gestion.location':'gestion.estado.location'
                                                                ,'estadoInstalacion':'gestion.estado.estadoInstalacion'
                                                                ,'nombreInstitucionEd':'gestion.estado.nombreInstitucionEd'
                                                                ,'matricula':'gestion.estado.matricula'
                                                                ,'municipioPDET':'gestion.estado.municipioPDET'    
                                        ,'cod_servicio':'gestion.estado.cod_servicio'
                                        ,'codDaneMuni':'gestion.estado.codDaneMuni'
                                        ,'codCentroPoblado':'gestion.estado.codCentroPoblado'
                                        ,'codDaneInstitucionEdu':'gestion.estado.codDaneInstitucionEdu'
                                        ,'tipoSitio':'gestion.estado.tipoSitio'
                                        ,'detalleSitio':'gestion.estado.detalleSitio'
                                        ,'energia':'gestion.estado.energia'
                                        ,'region':'gestion.estado.region'
                                        ,'grupoDesc' :'gestion.estado.grupoDesc'
                                        })
                
    completo["gestion.fechaControl"] =  ahora_cdd
    completo["gestion.fecha"]=  completo["gestion.fechaControl"].str.split(" ", n = 1, expand = True)[0]
    completo["gestion.hora"]=completo["gestion.fechaControl"].str.split(" ", n=1, expand=True)[1].str.split(":", n = 2, expand = True)[0]
    completo["gestion.anyo"] =  completo["gestion.fecha"].str[0:4]
    completo["gestion.mes"]  =  completo["gestion.fecha"].str[5:7]
    completo["gestion.dia"]  =  completo["gestion.fecha"].str[8:10]
    completo["gestion.totales.cantDev"]  = totales  
    completo["gestion.totales.cantDevConectados"]  =  conectados
    completo["gestion.totales.cantDevdesConectados"]  =  desconectados
    
    
except Exception as e:
    completo = pd.DataFrame(columns=['gestion.estado.nombreSede', 'gestion.estado.DDA','gestion.estado.estadoInstalacion', 'gestion.estado.COD_ISO',
   'gestion.estado.energia', 'gestion.estado.dptoGestion','gestion.estado.codCentroPoblado', 'gestion.estado.codDanesede',
   'gestion.estado.tipoSitio', 'gestion.estado.codDaneMuni','gestion.estado.nombre_centro_pob', 'gestion.estado.site_id',
   'gestion.estado.matricula', 'gestion.estado.energiadesc','gestion.estado.grupoDesc', 'gestion.estado.cod_servicio',
   'gestion.estado.region', 'gestion.estado.detalleSitio','gestion.estado.muniGestion', 'gestion.estado.id_Beneficiario',
   'gestion.estado.codDaneInstitucionEdu', 'gestion.estado.location','status', 'gestion.estadoCentro', 'itsm_incident_id', 'fechahora',
   'gestion.fechaControl', 'gestion.fecha', 'gestion.hora', 'gestion.anyo','gestion.mes', 'gestion.dia', 'gestion.totales.cantDev',
   'gestion.totales.cantDevConectados','gestion.totales.cantDevdesConectados'])
    print(e.message)


In [635]:
idx=completo[completo['gestion.estado.matricula']=='No aplica'].index

completo.loc[idx,'gestion.estado.matricula']=0

In [636]:
# insertando los datos en el indice 

use_these_keys = ['gestion.estado.id_Beneficiario'
                  ,'gestion.estadoCentro'
                  ,'gestion.estado.nombreSede'
                  ,'gestion.estado.COD_ISO'
                  ,'gestion.estado.codDanesede'
                  ,'gestion.estado.nombre_centro_pob'
                  ,'gestion.estado.site_id'
                  ,'gestion.estado.dptoGestion'
                  ,'gestion.estado.energiadesc'
                  ,'gestion.estado.muniGestion'
                  ,'gestion.estado.location'  
                  ,'gestion.estado.DDA'
                  ,'gestion.estado.nombreInstitucionEd'
                  ,'gestion.estado.matricula'
                  ,'gestion.estado.municipioPDET'    
                  ,'gestion.fechaControl'
                  ,'gestion.fecha'
                  ,'gestion.anyo'
                  ,'gestion.mes'
                  ,'gestion.dia'
                  ,'gestion.hora'
                  ,'gestion.totales.cantDev'
                  ,'gestion.totales.cantDevConectados'
                  ,'gestion.totales.cantDevdesConectados'                   
                  ,'gestion.estado.cod_servicio'
                  ,'gestion.estado.codDaneMuni'                  
                  ,'gestion.estado.codCentroPoblado'
                  ,'gestion.estado.codDaneInstitucionEdu'
                  ,'gestion.estado.tipoSitio'
                  ,'gestion.estado.detalleSitio'
                  ,'gestion.estado.energia'
                  ,'gestion.estado.region'
                  ,'gestion.estado.grupoDesc'
                  ,'gestion.estado.estadoInstalacion'
                  ,'@timestamp']                         

In [597]:
completo['@timestamp'] = now.isoformat()
def doc_generator2(df,use_these_keys):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'Estado-'+str(+document['gestion.estado.id_Beneficiario']) + '-' + str(document['gestion.fechaControl'])}",
                "_source": filterKeys(document,use_these_keys),
            }

salida = helpers.bulk(es, doc_generator2(completo,use_these_keys))
print("Fecha: ", now,"- Llamadas insertadas en indice principal:",salida[0])

Fecha:  2021-07-15 10:22:18.276125 - Llamadas insertadas en indice principal: 1193
